# Tool Calling

In [1]:
import dotenv
from agents import Agent, ModelSettings, Runner, function_tool, trace

dotenv.load_dotenv()

True

Create a static calorie table that we can use as a tool:

In [4]:
@function_tool
def get_food_calories(food_item: str) -> str:
    """
    Get calorie information for common foods to help with nutrition tracking.

    Args:
        food_item: Name of the food (e.g., "apple", "banana")

    Returns:
        Calorie information per standard serving
    """
    # Simple calorie database - in real world, you'd use USDA API
    calorie_data = {
        "apple": "80 calories per medium apple (182g)",
        "banana": "105 calories per medium banana (118g)",
        "broccoli": "25 calories per 1 cup chopped (91g)",
        "almonds": "164 calories per 1oz (28g) or about 23 nuts",
    }

    food_key = food_item.lower()
    if food_key in calorie_data:
        return f"{food_item.title()}: {calorie_data[food_key]}"
    else:
        return f"I don't have calorie data for {food_item} in my database. Try common foods like apple, chicken breast, or rice."

Let's test this out: 

_The following cell only works before you add the `@function_tool` annotation to `get_food_calories` function_

In [5]:
# get_food_calories('banana')

In [6]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    """,
    tools=[get_food_calories],
)

In [7]:
with trace("Nutrition Assistant with tools"):
    result = await Runner.run(
        calorie_agent, "How many calories are in total in a banana and an apple?"
    )
    print(result.final_output)

About 185 calories total (banana ~105 cal + apple ~80 cal for a medium fruit).


Enforce tools use:

In [8]:
calorie_agent = Agent(
    name="Nutrition Assistant",
    instructions="""
    You are a helpful nutrition assistant giving out calorie information.
    You give concise answers.
    """,
    tools=[get_food_calories],
    model_settings=ModelSettings(tool_choice="get_food_calories"),
)

with trace("Nutrition Assistant with tools enforced"):
    result = await Runner.run(
        calorie_agent, "How many calories are in total in a banana and an apple?"
    )
    print(result.final_output)

Error getting response: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-5-nano in organization org-vXYG8Tr6kanJtDxBAjNjuUCV on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}. (request_id: req_c00a2c14edea4740b02a8ebe1d9003e4)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for gpt-5-nano in organization org-vXYG8Tr6kanJtDxBAjNjuUCV on requests per min (RPM): Limit 3, Used 3, Requested 1. Please try again in 20s. Visit https://platform.openai.com/account/rate-limits to learn more. You can increase your rate limit by adding a payment method to your account at https://platform.openai.com/account/billing.', 'type': 'requests', 'param': None, 'code': 'rate_limit_exceeded'}}